# Learning Outcomes
- Quantify the image segmentation performance: Intersection over Union (IOU).
- Image gradients -> edge detection (image segmentation method)
- Contour (curves joining the boundaries of object (homogenous regions))
- Contour properties and features (area, perimeter, center of mass, bounding box).
- Blob detection (if manage to cover)


## Setup

In [ ]:
!pip install opencv-contrib-python

In [ ]:
!pip install requests


In [ ]:
import sys
assert sys.version_info >= (3, 7)

import numpy as np
import cv2 as cv
from util_func import *

### IOU
Formulas:



In [ ]:
def computeIOU(boxA, boxB):
    """Args:
    It should be(x1, y1, x2, y2)"""
    x_start = max(boxA[0], boxB[0])
    y_start = max(boxA[1], boxB[1])
    x_end = min(boxA[2], boxB[2])
    y_end = min(boxA[3], boxB[3])
    
    interArea = max(0, x_end - x_start + 1) * max(0, y_end - y_start + 1)
    
    # area of A and area of B
    areaA = (boxA[2] - boxA[1] + 1) * (boxA[3] - boxA[1] + 1)
    areaB = (boxB[2] - boxB[1] + 1) * (boxB[3] - boxB[1] + 1)
    
    return interArea / (areaA + areaB - interArea)

In [ ]:
img = cv.imread("image/lena.jfif")

In [ ]:
boxes = cv.selectROIs("boxes", img, showCrosshair = False)

cv.waitKey(0)
cv.detroyAllWindows()

In [ ]:
boxes

In [ ]:
def conver_xywh_to_xyzy(box):
    return [box[0], box[1], box[0] + box[2], box[1] + box[3]]

In [ ]:
gt = convert_xywh_to_xyzy(boxes[0])
pred = convert_xywh_to_xyzy(boxes[1])

img_copy = img.copy()
cv.rectangle(img_copy, (gt[0], gt[1]), (gt[2], gt[3]), (0, 0, 225), 1)
cv.rectangle(img_copy, (pred[0], pred[1]), (pred[2], pred[3]), (255, 0, 20), 1)
cv.putText(img_copy, f"IOU: {computeIOU(gt, pred):.3f}", (10, 25), 
           cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

show_img("IOU", img_copy)

### Image gradient / edge detecion
One of the most operators: Sobel. At the backend, convolution with specific kernel:

$$\begin{matrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{matrix}$$

### caveat (bit depth)


In [ ]:
img = cv.imread("images/wood_planck.jfif", 0)

th = cv.threshold(img, 200, 255, cv.THRES_BINARY_INV)[1]

show_img("binary", th)

In [ ]:
sobelx_8u = cv.Sobel(th, cv.CV_8U, 1, 0)

# correct way
sobelx_32f = cv.Sobel(th, cv.CV_32F, 1, 0)
sobelx = cv.convertScaleAbs(sobelx_32f)

plt.subplot(121), plt_img(sobelx_8u, "CV_8U")
plt.subplot(122), plt_img(sobelx, "CV_32F")
plt.show()

### Contstruct gradient map

In [ ]:
img = cv.imread("images/chessboard.png", 0)

# apply sobel x and sobel y
sobelx = cv.Sobel(img, cv_32F, 1, 0)
sobelx_8u= cv.convertScaleAbs(sobelx)
sobely = cv.Sobel(img, cv_32F, 0, 1)
sobely_8u = cv.convertScaleAbs(sobely)

# gradient
gradient = cv.magnitude(sobelx, sobely)
# direction
direction = np.arctan2(sobelx, sobely) * 180 / np.pi % 100

plt.subplot(221), plt_img(sobelx_8u, "vertical")
plt.subplot(222), plt_img(sobely_8u, "horinzontal")
plt.subplot(223), plt_img(gradiet, cmap = "jet"), plt.title("gradient"), plt.colorbar()
plt.subplot(224), plt_img(direction, cmap = "jet"), plt.title("direction"), plt.colorbar()
plt.show()

### Canny edge detectors
- Enhance accuracy by reducing false positives
- Flexible

In [ ]:
img = cv.imread("images/chair.jpg", 0)

edge = cv.Canny(img, 100, 300)

plt.subplot(121), plt_img(img, "grayscale")
plt.subplot(122), plt_img(edge, "Canny")
plt.show()

In [ ]:
img = cv.imread("images/chair.jpg", 0)

edge = cv.Canny(img, 30, 150)

plt.subplot(121), plt_img(img, "grayscale")
plt.subplot(122), plt_img(edge, "Canny")
plt.show()

In [ ]:
# simple example: adjust one parameter:threshold1
img = cv.imread("images/bridge.jfif")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

ratio = 2.5
ksize = 3
wn = "Canny"
trackbarName = "Threshold"

def cannyThreshold(val):
    """val is Threshold"""
    edge = cv.Canny(gray, val, val * ration, apertureSize = ksize)
    # create mask
    mask = edge != 0
    res = img * (mask[:, :, None].astype(np.uint8))
    cv.imshow(wn, res)
    
cv.namedWindow(wn)
cv.createTrackbar(trackbarName, wn, 10, 100, cannyThreshold)

cv.waitKe(0)
cv.destroyAllWindows()

In [ ]:
def auto_canny(img, method, sigma = 0.33):
    """Args:
    img: grayscale image
    method: median, otsu, triangle
    sigma: 0.33 (default)"""
    if method == "median":
        Th = np.median(img)
    
    elif method == "triangle":
        Th = cv.thresholding(img, 0, 255, cv.THRES_TRIANGLE)[0]
        
    elif method == "otsu":
        Th = cv.thresholding(img, 0, 255, cv.THRES_OTSU)[0]
        
    Thresh1 = (1 - sigma) * Th
    Thresh2 = (1 + sugma) * Th
    
    return cv.Canny(img, Thresh1, Thresh2)

### Contour detection
1. Read an image
2. Threshold / Edge detection
3. The output from step 2 can be parse into `cv.findContour()`.
4. (optional) draw contour, cv.drawContour()

In [ ]:
rect = np.zeros((256, 256), dtype = np.uint8)

cv.rectangle(rect, (25, 25), (231, 231), 255, -1)

show_img("rectangle", rect)

In [ ]:
contours, _ = cv.findContour(rect, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

print(contours)

In [ ]:
type(contours)

In [ ]:
contours[0].shape

In [ ]:
img_bgr = cv.cvtColor(rect, cv.COLOR_GRAY2BGR)

cv.drawContours(img_bgr contours, -1, (0, 255, 0), 2)

show_img("contours", img_bgr)

In [ ]:
img = cv.imread("images/monitor.jfif", 0)

th = cv.thresholding(img, 200, 255, cv.THRESH_BINARY_INV)[1]

#contour
contours, _ = cv.findContour(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
cv.drawContours(img_bgr, contours, -1, (-, 255, 0), 2)

show_img("contour", img_bgr)

In [ ]:
len(contours)

In [ ]:
contours, _ = cv.findContour(th, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
cv.drawContours(img_bgr, contours, -1, (-, 255, 0), 2)

show_img("contour", img_bgr)

In [ ]:
# simple way to sift through contours
contours, _ = cv.findContour(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

#select the contour that has the highest number of points
length = [len(c) for c in contours]
cnt = contours[np.argmax(length)]

img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
cv.drawContours(img_bgr, [cnt], -1, (-, 255, 0), 2)

show_img("contour", img_bgr)

### Contour features
- area
- perimeter
- centroid
- bounding box

In [ ]:
M = cv.moments(cnt)
print(M)

In [ ]:
# centroid
cx = int(M['m10'] / M['m00'])
cy = int(M['m01'] / M['m00'])
print(f"The centroid of monitor conour: {(cx, cy)}.")

In [ ]:
# area and perimeter
area = cv.contourArea(cnt)
peri = cv.arcLength(cnt, True)

print(f"area of monitor contour: {area}")
print(f"perimeter of monitor contour: {peri:.3f}")

In [ ]:
# apply edge detection and contour properties to segment
img = cv.imread("images/remote-controller.webp")

In [ ]:
print(cv.__version__)

In [ ]:
# resize ->grayscale -> bilateral filter -> edge (canny) -> area -> DP approximation
factor = 300 / im.shape[1]
img = cv.resize(img, None, fx = factor, fy = factor)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
blur = cv.bilateralFilter(gray, 7, 19, 13)
edge = auto_canny(blur, method = "triangle")

show_img("edge", edge)

In [ ]:
contours, _ = cv.findContours(edge, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

#sort the contour with respect to contour area in descending order, and grab the first 5 largest contours
contours = sorted(contours, key = cv.contourArea, reverse = True)[:5]

for c in contours:
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.1 * peri, True)
    
    if len(approx) == 4:
        screen = c
        break
        
img_copy = img.copy()
cv.drawContours(img, [screen], -1, (0, 255, 2), 2)
show_img("contour", img_copy)

### Additional Contour Properties
$$circularity = \frac{ 4 * pi * Area }{ perimeter ** 2 }$$

### Demo on red color segmentation

In [ ]:
redLow1 = (0, 90, 40)
redHigh1 = (10, 255, 210)

redLow2 = (170, 90, 40)
redHigh2 = (179, 255, 210)

cap = cv.VideoCapture(0)

if not cap.isOpened():
    sys.exit("No webcam detected")

fixed_width = min_area = 500
factor = fixed_width / cap.get(3)  # frame widhth
kernel = np.ones((3, 3), dtype = np.uint8)

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("No frame received")
        break
        
    # resize -> blurring -> change to hsv -> mask integration -> morphological operations ->
    # find contours -> bounding box
    resized = cv.resize(frame, None, fx = factor, fy = factor)
    blur = cv.GaussianBlur(resized, (5, 5), 0)
    img_hsv = cv.cvtColor(blur, COLOR_BGR2HSV)
    
    mask1 = cv.inRange(img_hsv, redLow1, redHigh1)
    mask2 = cv.inRange(img_hsv, redLow2, redHigh2)
    # Opening to remove noises
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel, iteration = 2)
    
    # contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    bbs = []
    
    for c in contours:
        area = cv.contourArea(c)
        
        if area > min_area:
            bb = cv.boundinRect(c)
            bbs.append(bb)
            M = cv.moments(c)
            cx, cy = int(M["m10"] / M["m00"]), int(M["m01"] / M["mm00"])
            centroids.append((cx, cy))
    
    for bb, centroids in zip(bbs, centroids):
        x, y, w, h = bb
        cv.rectangle(resized, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv.circle(resized, centroid, 2, (255, 0, 0), -1)
        
    cv.imshow("red object", resized)
    k = cv.waitkey(10) & 0xFF
    if k == 27:
        break

cv.destroyAllWindows()
cap.release()

### Exercise 

In [ ]:
# Question 1
#sobel
img = cv.imread("images/pineapple.jfif", 0)

show_img("img", img)

sobelx_32f = cv.Sobel(img, cv.CV_32F, 1, 0, ksize = 3)
sobely_32f = cv.Sobel(img, cv.CV_32F, 0, 1, ksize = 3)

sobel = np.sqrt(sobelx_32f ** 2 + sobely_32f ** 2).astype(np.uint8)

In [ ]:
# Laplacian
laplacian = cv.Laplacian(img, cv.CV_32F, ksize=3)
show_img("img", laplacian)

In [ ]:
# Canny
canny = cv.Canny(img, 100, 200)

In [ ]:
plt.figure(figsize = (14, 12))

plt.subplot(321), plt_img(sobel, "Sobel")
plt.subplot(322), plt_img(laplacian, "Laplacian")
plt.subplot(323), plt_img(canny, "Canny")

plt.show()

In [ ]:
# Question 2
img = cv.imread("images/electronic.jfif")
show_img("img", img)
img_copy = img.copy()

gray = cv.cvtColor(img_copy, cv.COLOR_RGB2GRAY)

threshold = cv.threshold(gray, 183, 255, cv.THRESH_BINARY)[1]

contours, _ = cv.findContours(threshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

# draw contours
for c in contours:
    x, y, w, h = cv.boundingRect(c)
    if w>5 and h>5: 
        cv.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
# Display
plt.figure(figsize = (5, 2))
plt.imshow(cv.cvtColor(img_copy,cv.COLOR_RGB2BGR))
plt.title("White Object")
plt.show()

In [ ]:
# Question 3
img = cv.imread("images/clock.jpg")
show_img("img", img)

blur = cv.GaussianBlur(img, (3, 3), 0)
gray = cv.cvtColor(blur, cv.COLOR_BGR2GRAY)
edge = cv.Canny(gray, 50, 150)

show_img("edge", edge)

# find contour
contours, _ = cv.findContours(edge, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

contours = sorted(contours, key=cv.contourArea, reverse = True)[:5]
clk = None

for c in contours:
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.1*peri, True)
    
    if len(approx) == 4:
        clk = c
        break
        
img_copy = img.copy()
cv.drawContours(img_copy, [clk], -1, (0, 255, 0), 2)

show_img("Clock", img_copy)